# the machinery needed to use markdown, pug, and jinja in notebooks

this notebook grew out of hand. it now handles creating templates in notebooks,
and reusing those templates. further it provides tempalte interoperability between
jinja2, markdown, and pug.

we expose these templates in the notebook that is saved on disk.
then test a `NotebookLoader` that can extract templates in the notebook outputs.

In [1]:
%%
## `get_environment` creates our extended environment

    def get_environment(shell, markdown_env={}):
a `jinja2.Environment` for ipython that makes templates reusable inside and outside notebooks
        
        import jinja2.ext, pypugjs.ext.jinja, markdown_it
        env = Environment(loader=jinja2.ChoiceLoader([jinja2.FileSystemLoader("")]))
initialize the environment with an extensible `jinja2.ChoiceLoader` and one that lets
you access template naturally from the filesystem. the globals are shared with the interactive namespace
allowing variables to be used to in templates

        env.extend(dict_loader=jinja2.DictLoader({}))
we add dict loader that lets us add arbitrary templates to our environment.
this is primarily used by the `%template` magic for creating and using templates

        shell.add_traits(templates=traitlets.Instance(jinja2.DictLoader, ({},), {}))
        env.loader.loaders.append(env.dict_loader)
we expose this loader in the `env.loader` and in the `IPython` shell
    
        env.filters["md"] = partial(markdown_it.MarkdownIt().render, env=markdown_env)
initialize a markdown filter that can be used in jinja2...

        env.add_extension(pypugjs.ext.jinja.PyPugJSExtension)
...and now as `pug` filters with the `pypugjs` extension installed.

        env.filters["pug"] = pugjs
it makes sense to expose `pug` as a filter in jinja

        pypugjs.register_filter("md")(env.filters["md"])
and attached markdown filter to the `pypug` registry`

        shell.register_magic_function(template_magic, "line_cell", "template")
all of this is easier to manage as an `IPython` `%template` magic
        
        return env

## `get_environment` creates our extended environment

    def get_environment(shell, markdown_env={}):
a `jinja2.Environment` for ipython that makes templates reusable inside and outside notebooks
        
        import jinja2.ext, pypugjs.ext.jinja, markdown_it
        env = Environment(loader=jinja2.ChoiceLoader([jinja2.FileSystemLoader("")]))
initialize the environment with an extensible `jinja2.ChoiceLoader` and one that lets
you access template naturally from the filesystem. the globals are shared with the interactive namespace
allowing variables to be used to in templates

        env.extend(dict_loader=jinja2.DictLoader({}))
we add dict loader that lets us add arbitrary templates to our environment.
this is primarily used by the `%template` magic for creating and using templates

        shell.add_traits(templates=traitlets.Instance(jinja2.DictLoader, ({},), {}))
        env.loader.loaders.append(env.dict_loader)
we expose this loader in the `env.loader` and in the `IPython` shell
    
        env.filters["md"] = partial(markdown_it.MarkdownIt().render, env=markdown_env)
initialize a markdown filter that can be used in jinja2...

        env.add_extension(pypugjs.ext.jinja.PyPugJSExtension)
...and now as `pug` filters with the `pypugjs` extension installed.

        env.filters["pug"] = pugjs
it makes sense to expose `pug` as a filter in jinja

        pypugjs.register_filter("md")(env.filters["md"])
and attached markdown filter to the `pypug` registry`

        shell.register_magic_function(template_magic, "line_cell", "template")
all of this is easier to manage as an `IPython` `%template` magic
        
        return env

In [2]:
MARKDOWN_EXT = ".md",

In [3]:
%%  
### an overloaded jinja2 environment

    class Environment(jinja2.Environment):
the best case scenario is a jinja environment that handles all the output reasoing.
        
        class template_class(jinja2.Template):
templates are the only way to control the output type of a template.
so we render notebooks in ipython with a special `template_class` that exports html.
_it would be possible extend this to an updating display._
    
            def render(self, *args, **kwargs):
                text = super().render(*args, **kwargs)
                if (self.name or self.filename or "").endswith(".md"):
                    text = self.environment.filters["md"](text)
                return text

        def from_string(self, source, globals = None, template_class = None, filename = None, name=None):
we want to be able to dispatch different renderers based on the filename.
the default `Environment.from_string` does not accept any argument so we customize
the method to accept it. now the filename is carried through to the template and we can
reason on the `Template.name`.
        
            gs = self.make_globals(globals)
            cls = template_class or self.template_class
            return cls.from_code(self, self.compile(source, name=name, filename=filename), gs, None)

### an overloaded jinja2 environment

    class Environment(jinja2.Environment):
the best case scenario is a jinja environment that handles all the output reasoing.
        
        class template_class(jinja2.Template):
templates are the only way to control the output type of a template.
so we render notebooks in ipython with a special `template_class` that exports html.
_it would be possible extend this to an updating display._
    
            def render(self, *args, **kwargs):
                text = super().render(*args, **kwargs)
                if (self.name or self.filename or "").endswith(".md"):
                    text = self.environment.filters["md"](text)
                return text

        def from_string(self, source, globals = None, template_class = None, filename = None, name=None):
we want to be able to dispatch different renderers based on the filename.
the default `Environment.from_string` does not accept any argument so we customize
the method to accept it. now the filename is carried through to the template and we can
reason on the `Template.name`.
        
            gs = self.make_globals(globals)
            cls = template_class or self.template_class
            return cls.from_code(self, self.compile(source, name=name, filename=filename), gs, None)

In [4]:
parser = argparse.ArgumentParser("template")
parser.add_argument("name", nargs='?', default=None)
parser.add_argument("-w", "--write", action="store_true");

In [5]:
%%
## an ipython magic
    def template_magic(line, cell):
a template magic that will save templates, and render the templates as html

        args = parser.parse_args(shlex.split(line))    
        if cell is None:
            return
        if args.name:
            if cell is None:
                return shell.env.get_template(args.name).render(globals())
            shell.env.dict_loader.mapping[args.name] = cell
            if args.write:
                Path(args.name).write_text(cell)
            return display(
                HTML(shell.env.from_string(cell, name=line).render(globals())), metadata={"jinja2:Template": {line: cell}}
            )
        return shell.env.get_template(line).render(globals())

## an ipython magic
    def template_magic(line, cell):
a template magic that will save templates, and render the templates as html

        args = parser.parse_args(shlex.split(line))    
        if cell is None:
            return
        if args.name:
            if cell is None:
                return shell.env.get_template(args.name).render(globals())
            shell.env.dict_loader.mapping[args.name] = cell
            if args.write:
                Path(args.name).write_text(cell)
            return display(
                HTML(shell.env.from_string(cell, name=line).render(globals())), metadata={"jinja2:Template": {line: cell}}
            )
        return shell.env.get_template(line).render(globals())

In [6]:
%%
## a pug filter running through jinja

    def pugjs(source):
        return shell.env.from_string(env.extensions["pypugjs.ext.jinja.PyPugJSExtension"].preprocess(
            source,  "this.pug"
        ), None, HtmlTemplate).render()

## a pug filter running through jinja

    def pugjs(source):
        return shell.env.from_string(env.extensions["pypugjs.ext.jinja.PyPugJSExtension"].preprocess(
            source,  "this.pug"
        ), None, HtmlTemplate).render()

In [7]:
%%
## the `IPython` extension

    def load_ipython_extension(shell):
        env = get_environment(shell)
        shell.add_traits(env=traitlets.Instance("jinja2.Environment", (), {}, default_value=env))
        shell.env = env


    if __name__ == "__main__":
        load_ipython_extension(shell)

## the `IPython` extension

    def load_ipython_extension(shell):
        env = get_environment(shell)
        shell.add_traits(env=traitlets.Instance("jinja2.Environment", (), {}, default_value=env))
        shell.env = env


    if __name__ == "__main__":
        load_ipython_extension(shell)

## modding `pypugjs` jinja opinions

In [8]:
    def visitInclude(self, node):
        if md := node.path.endswith(MARKDOWN_EXT):
            self.buf.append("{% filter md%}")
        self.buf.append("{% include '")
        self.buf.append(node.path)
        self.buf.append("'%}")
        if md:
            self.buf.append("{% endfilter %}")

    pypugjs.ext.jinja.Compiler.visitInclude = visitInclude

In [9]:
%%file external.pug
figure
    figcaption this is created from a pug template and can accept variables

Overwriting external.pug


In [10]:
%%template pug_in_markdown.md
#### using external pug in markdown

{% include "external.pug" %}

In [11]:
%%file external.md
_external markdown included through jinja._

Overwriting external.md


In [12]:
%%template markdown_in_pug.pug
figure
    figcaption using external markdown in pug
    include external.md

In [13]:
%%
## loading jinja templates from as notebook

    @dataclass(unsafe_hash=True)
    class NotebookLoader(jinja2.BaseLoader):
        path: Path
        def get_templates_from_notebook_output(self):
            return get_templates_from_notebook_output(self.path)
        def list_templates(self):
            return list(self.get_templates_from_notebook_output())
        def get_source(self, environment, name, globals=None):
            t = Path(self.path).stat().st_mtime
            try:
                return self.get_templates_from_notebook_output()[name], name, lambda: Path(self.path).stat().st_mtime == t
            except:
                raise jinja2.TemplateNotFound("can't find template")
    shell.env.loader.loaders = shell.env.loader.loaders[:2]

    def get_templates_from_notebook_output(file):
        return (
            pandas.read_json(file, typ=Series)[["cells"]]
                .apply(Series).stack().apply(Series)
            [["outputs"]].dropna().stack().apply(Series)
            .dropna().stack().apply(Series)
            [["metadata"]].stack().apply(Series)[["jinja2:Template"]].stack().pipe(merge)
        )

    

## loading jinja templates from as notebook

    @dataclass(unsafe_hash=True)
    class NotebookLoader(jinja2.BaseLoader):
        path: Path
        def get_templates_from_notebook_output(self):
            return get_templates_from_notebook_output(self.path)
        def list_templates(self):
            return list(self.get_templates_from_notebook_output())
        def get_source(self, environment, name, globals=None):
            t = Path(self.path).stat().st_mtime
            try:
                return self.get_templates_from_notebook_output()[name], name, lambda: Path(self.path).stat().st_mtime == t
            except:
                raise jinja2.TemplateNotFound("can't find template")
    shell.env.loader.loaders = shell.env.loader.loaders[:2]

    def get_templates_from_notebook_output(file):
        return (
            pandas.read_json(file, typ=Series)[["cells"]]
                .apply(Series).stack().apply(Series)
            [["outputs"]].dropna().stack().apply(Series)
            .dropna().stack().apply(Series)
            [["metadata"]].stack().apply(Series)[["jinja2:Template"]].stack().pipe(merge)
        )

    

In [14]:
a = 1

## demos

### using a pug template

In [15]:
%%template this.pug
hgroup
    h4 a pug template from a notebook !{a}

### using a markdown template

In [16]:
%%template this.md
#### a markdown template in a notebook

{{a}}

## tests

In [18]:
    >>> env = Environment(loader=NotebookLoader("2025-09-11-jinja.ipynb"))
    ... env.add_extension("pypugjs.ext.jinja.PyPugJSExtension")
    ... env.filters.update(shell.env.filters)
    ... env.globals = globals()
    >>> assert env.get_template("this.pug").render(a="abc").startswith("<hgroup")


## marginalia

with pidgy, i realized that having markdown cell inputs and markdown output is redundant.
for simplicty, i've been using this technique, but the real win is to have html in and markdown out.

it should be possible load templates for notebooks. 
it would be a notebook loader that took a notebook path and could discover templates
in the outputs.
